In [ ]:
#how csv are read in google collaboratory - https://stackoverflow.com/questions/48340341/read-csv-to-dataframe-in-google-colab
from google.colab import drive 
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [ ]:
%cd "gdrive/My Drive/Deep_Text_Corrector"

/content/gdrive/My Drive/Deep_Text_Corrector


In [ ]:
pip install python-docx

     |████████████████████████████████| 5.5MB 2.9MB/s 
  Created wheel for python-docx: filename=python_docx-0.8.10-cp36-none-any.whl size=184491 sha256=afbdecb6ec98b3e370c151b9c53bf2452b3690c064f3070e7ec1f83c1d802273
  Stored in directory: /root/.cache/pip/wheels/18/0b/a0/1dd62ff812c857c9e487f27d80d53d2b40531bec1acecfa47b
Successfully built python-docx


In [ ]:
from docx import Document
from tqdm import tqdm_notebook as tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import random
import spacy
import re
import string
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import coo_matrix
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from tensorflow.keras.layers import LSTM, GRU, Dense, Input, Embedding
from tensorflow.keras.models import Model
from numpy import array
from sklearn.utils import shuffle
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Lambda
from tensorflow.keras.backend import concatenate

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
tf.config.experimental.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

__Data Preprocessing__

In [ ]:
#https://automatetheboringstuff.com/chapter13/
doc = Document('Harry_Potter.docx')
len(doc.paragraphs)

91663

In [ ]:
#Getting paragraphs from the doc
paras=[]
for para in doc.paragraphs:
  if para.text != '':
    paras.append(para.text.strip())

In [ ]:
len(paras)

41053

In [ ]:
paras2=[]
skip=0
for para in paras:
  if 'CHAPTER' in para:
    skip+=2
  if skip: 
    skip-=1
  else:
    paras2.append(para)

In [ ]:
len(paras2)

40657

In [ ]:
#removing paragraphs with missing quatation endings.
new_paras=[]
count=0
for para in paras2:
  if para.count('“')!=para.count('”'):
    count+=1
    continue
  new_paras.append(para)
count

1685

In [ ]:
len(new_paras)

38972

In [ ]:
new_paras_2=new_paras[:30000]

In [ ]:
total_text=" ".join(new_paras_2)

In [ ]:
” “ ’

In [ ]:
#sentence detection - https://realpython.com/natural-language-processing-spacy-python/
def set_custom_boundaries(doc):
  for i,token in enumerate(doc[:-1]):
    if token.text == ')' and doc[i-1].text == '.' and i>0:
      doc[i+1].is_sent_start = True
    elif token.text ==  ';' and not doc[i+1].text.islower():
      doc[i+1] .is_sent_start  = True
    elif token.text ==  '!' and not doc[i+1].text.islower():
      doc[i+1] .is_sent_start  = True
    elif token.text ==  '?' and not doc[i+1].text.islower():
      doc[i+1] .is_sent_start  = True
    elif token.text == '”' and (doc[i-1].text=='.' or doc[i-1].text=='?' or doc[i-1].text=='!' or doc[i-1].text==';') and i>0 and not doc[i+1].text.islower():
      doc[i+1].is_sent_start = True
    elif token.text == '.' and doc[i+1].text==')':
      doc[i+1].is_sent_start = False
    elif token.text == '.' and doc[i+1].text!='.' and not doc[i+1].text.islower():
      doc[i+1].is_sent_start = True
    else:
      doc[i+1].is_sent_start  = False
  return doc

lines=[]
custom_nlp = spacy.load('en_core_web_sm')
custom_nlp.max_length = 4579094
custom_nlp.add_pipe(set_custom_boundaries, before='parser')
custom_doc = custom_nlp(total_text)

In [ ]:
lines=[]
custom_sentences = list(custom_doc.sents)
for line in custom_sentences:
  lines.append(str(line))

In [ ]:
len(lines)

73452

In [ ]:
lines[0:10]

['Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say that they were perfectly normal, thank you very much.',
 'They were the last people you’d expect to be involved in anything strange or mysterious, because they just didn’t hold with such nonsense.',
 'Mr. Dursley was the director of a firm called Grunnings, which made drills.',
 'He was a big, beefy man with hardly any neck, although he did have a very large mustache.',
 'Mrs. Dursley was thin and blonde and had nearly twice the usual amount of neck, which came in very useful as she spent so much of her time craning over garden fences, spying on the neighbors.',
 'The Dursleys had a small son called Dudley and in their opinion there was no finer boy anywhere.',
 'The Dursleys had everything they wanted, but they also had a secret, and their greatest fear was that somebody would discover it.',
 'They didn’t think they could bear it if anyone found out about the Potters.',
 'Mrs. Potter was Mrs. Dursley’s sister, but

In [ ]:
#replacing tab spaces with single space
lines_2=[]
for line in lines:
  if '\t' in line:
    line=line.replace('\t',' ')
    lines_2.append(line)
  else:
    lines_2.append(line)

In [ ]:
#sentences with quotations - get sentences with proper quotations to get linguistically meaningful units
skip=0
new_lines=[]
for ind,line in enumerate(lines_2):
  if skip:
      skip -= 1
      continue
  #if line.count('"')%2!=0:
  #if ('”' in line and '“' not in line) or ('“' in line and '”' not in line):
  if line.count('“')!=line.count('”'):
    l=""
    l=l+lines_2[ind]
    skip=0
    #while(l.count('"')%2!=0):
    #while('”' in l and '“' not in l) or ('“' in l and '”' not in l):
    while l.count('“')!=l.count('”'):
      if (lines_2[ind+1]=='”'):
        l=l+'”'
      elif (lines_2[ind+1].startswith('”') or l=='“'):
        l=l+lines_2[ind+1]
      else:
        l=l+" "+lines_2[ind+1]
      ind+=1
      skip+=1
    new_lines.append(l)
  else:
    new_lines.append(line)

In [ ]:
new_lines[0:10]

['Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say that they were perfectly normal, thank you very much.',
 'They were the last people you’d expect to be involved in anything strange or mysterious, because they just didn’t hold with such nonsense.',
 'Mr. Dursley was the director of a firm called Grunnings, which made drills.',
 'He was a big, beefy man with hardly any neck, although he did have a very large mustache.',
 'Mrs. Dursley was thin and blonde and had nearly twice the usual amount of neck, which came in very useful as she spent so much of her time craning over garden fences, spying on the neighbors.',
 'The Dursleys had a small son called Dudley and in their opinion there was no finer boy anywhere.',
 'The Dursleys had everything they wanted, but they also had a secret, and their greatest fear was that somebody would discover it.',
 'They didn’t think they could bear it if anyone found out about the Potters.',
 'Mrs. Potter was Mrs. Dursley’s sister, but

In [ ]:
len(new_lines)

48901

In [ ]:
#sentences with braces - get sentences with proper opening and closing braces to get linguistically meaningful units
skip=0
new_lines_2=[]
for ind,line in enumerate(new_lines):
  if skip:
      skip -= 1
      continue
  #if '(' in line and ')' not in line:
  if line.count('(')!=line.count(')'):
    l=""
    l=l+new_lines[ind]
    skip=0
    while(')' not in l):
      if (new_lines[ind+1]==')'):
        l=l+')'
      elif (new_lines[ind+1].startswith(')') or l=='('):
        l=l+new_lines[ind+1]
      #if(new_lines[ind+1].startswith(')')):
      #  l=l+new_lines[ind+1]
      else:
        l=l+" "+new_lines[ind+1]
      ind+=1
      skip+=1
    new_lines_2.append(l)
  else:
    new_lines_2.append(line)

In [ ]:
len(new_lines_2)

48818

In [ ]:
new_lines_2[-10:]

['“Thus far, as I hope you agree, I have shown you reasonably firm sources of fact for my deductions as to what Voldemort did until the age of seventeen?”',
 'Harry nodded.',
 '“But now, Harry,” said Dumbledore, “now things become murkier and stranger. If it was difficult to find evidence about the boy Riddle, it has been almost impossible to find anyone prepared to reminisce about the man Voldemort. In fact, I doubt whether there is a soul alive, apart from himself, who could give us a full account of his life since he left Hogwarts. However, I have two last memories that I would like to share with you.”',
 'Dumbledore indicated the two little crystal bottles gleaming beside the Pensieve.',
 '“I shall then be glad of your opinion as to whether the conclusions I have drawn from them seem likely.”',
 'The idea that Dumbledore valued his opinion this highly made Harry feel even more deeply ashamed that he had failed in the task of retrieving the Horcrux memory, and he shifted guiltily in

In [ ]:
#removing sentences which has only one word
new_lines_3=[]
for line in new_lines_2:
  if len(line.split())<2:
    pass
  else:
    if line.startswith(' '):
      line=line.replace(' ',"",1)
    line=line.replace('  ',' ')
    new_lines_3.append(line)

In [ ]:
len(new_lines_3)

48376

In [ ]:
#replacing ’ ‘ with ' - to get correct pos tags for 'd,'ve,n't etc,.
new_lines_4=[]
for line in new_lines_3:
  if "’" in line:
    line=line.replace("’","'")
  if "‘" in line:
    line=line.replace("‘","'")
  if "on'y" in line:
    line=line.replace("on'y","only")
  new_lines_4.append(line)

In [ ]:
len(new_lines_4)

48376

In [ ]:
df=pd.DataFrame(new_lines_4)
df.columns=['lines']
df.head()

,lines
0,"Mr. and Mrs. Dursley, of number four, Privet D..."
1,They were the last people you'd expect to be i...
2,Mr. Dursley was the director of a firm called ...
3,"He was a big, beefy man with hardly any neck, ..."
4,Mrs. Dursley was thin and blonde and had nearl...


In [ ]:
df.shape

(48376, 1)

In [ ]:
#splitting data into train, cv and test
train, test = train_test_split(df,test_size=0.2,random_state=42)
train, cv = train_test_split(train,test_size=0.2,random_state=42)
print(train.shape)
print(cv.shape)
print(test.shape)

(30960, 1)
(7740, 1)
(9676, 1)


In [ ]:
” “ ’

In [ ]:
"""
words after splitting string and tokens in spacy nlp(string) are different
as spacy treats punctuations included in a word as seperate tokens
Eg: 
"'my name don is the 'was the man then".split()
output:
["'my", 'name', 'don', 'is', 'the', "'was", 'the', 'man', 'then']

for t in nlp("'my name don is the 'was the man then"):
  print(t)
output:
'
my
name
don
is
the
'
was
the
man
then

If we observe, apostrophe(') is considered as seperate token in spacy.
Here we are considering punctuations and defining errors. For example, replacing
comma with fullstop, removing opening double quote, etc.
But, we can't replace tokens in spacy.
So, we convert it into string first and find index of the token to be replaced.
Below function finds index, and replaces tokens with error.
"""

#function to replace tokens with errors
def replace_with_error(about_inc,index,to_replace,error):
  l=str(about_inc)
  g=[]
  for i,t in enumerate(about_inc):
    if(i==index) and to_replace==about_inc[index].text:
      idx=t.idx
    elif(i==index):
      idx=t.idx-to_replace.find(about_inc[index].text)
    lent=len(to_replace)
  skip=0
  for i,ch in enumerate(l):
    if skip:
      skip-=1
      continue
    if(i==idx):
      g.append(error)
      skip=lent-1
    else:
      g.append(ch)
  return ''.join(g)

#function to define errors
def get_incorrect_lines(df):
  incorrect_lines=[]
  words_end_with=('ch','ss','sh','x','zz')
  words_ends_with_cont=("'ve","'ll","'s","'m","'d","'re","'t")
  vowels=('ay','ey','iy','oy','uy')
  nlp = spacy.load('en_core_web_sm')
  for i in tqdm(range(len(df.values))):
    
    about_doc = nlp(df.values[i])
    about_inc=nlp(df.values[i])

    #counts for not repeating same error twice in a line
    count1=0;count2=0;count3=0;count4=0;count5=0;count6=0;count7=0;count8=0
    count9=0;count10=0;count11=0;count12=0;count13=0;count14=0;count15=0;count16=0

    #count for maintaining only two errors per line
    error_count=0

    skip=0
    previous_tokens=[]
    for j,token in enumerate(about_doc):
      #storing tokens
      previous_tokens.append(token.text)

      #getting index for tokens in incorrect lines(about_inc)
      for t in about_inc:
        if about_doc[j].text==t.text:
          index=t.i
          break

      if skip:
        skip -= 1
        continue

      #if same token is present twice in the line, skipping that token
      if about_doc[j].text in previous_tokens[0:-1]:
        skip=1
      if skip:
        skip -= 1
        continue

      #1. replacing Verb Past Tense(VBD) to Verb present tense: were --> are, saw --> see
      if  token.tag_ == 'VBD' and about_inc[index].text!=token.lemma_ and about_inc[index].text[0].islower() and count1==0 and error_count<=2:
        about_inc=nlp(replace_with_error(about_inc,index,about_inc[index].text,token.lemma_))
        count1=1
        

      #2. Subject-verb agreement: they go -->they goes,we watch  -->we watches
      elif token.tag_ ==  'VBP':
        for t in nlp(str(token)):
          if t.tag_ == 'VB':
            if about_inc[index].text.endswith(words_end_with) or about_inc[index].text == 'go' or about_inc[index].text == 'do' and count2==0 and error_count<=2:
                about_inc=nlp(replace_with_error(about_inc,index,about_inc[index].text,about_inc[index].text+'es'))
                count2=1
                
            elif about_inc[index].text.endswith(vowels) and count2==0:
              about_inc=nlp(replace_with_error(about_inc,index,about_inc[index].text,about_inc[index].text+'s'))
              count2=1
              
            elif about_inc[index].text.endswith('y') and count2==0:
              about_inc=nlp(replace_with_error(about_inc,index,about_inc[index].text,about_inc[index].text.replace('y','ies')))
              count2=1
              
            elif about_inc[index].text == 'be' and count2==0:
              about_inc=nlp(replace_with_error(about_inc,index,about_inc[index].text,'is'))
              count2=1
              
            elif about_inc[index].text.endswith('s') and count2==0:
              about_inc=nlp(replace_with_error(about_inc,index,about_inc[index].text,about_inc[index].text+'ses'))
              count2=1
              
            elif about_inc[index].text.endswith('z') and count2==0:
              about_inc=nlp(replace_with_error(about_inc,index,about_inc[index].text,about_inc[index].text+'zes'))
              count2=1
              

        #3. Missing verb: However, there are also a great number of people [who → who are] against
        if about_inc[index-1].tag_ == 'WP' and j!=0 and count3==0:
          about_inc=nlp(replace_with_error(about_inc,index,about_inc[index].text+" ",""))
          count3=1
          

      #4. Removing Articles
      elif about_inc[index].text=='a' or about_inc[index].text=='an' or about_inc[index].text=='the' and count4==0:
        about_inc=nlp(replace_with_error(about_inc,index,about_inc[index].text+" ",""))
        count4=1
        

      #5. Noun number: children --> child
      elif token.tag_  == 'NNS' and about_inc[index].text!=token.lemma_ and about_inc[index].text[0].islower() and count5==0:
        about_inc=nlp(replace_with_error(about_inc,index,about_inc[index].text,token.lemma_))
        count5=1
        

      #6. Noun possessive: carrier's --> carriers
      elif about_inc[index].text=="'s" and count6==0:
        about_inc=nlp(replace_with_error(about_inc,index,about_inc[index].text,'s'))
        count6=1
        

      #7. Pronoun reference: they --> he/she
      elif about_inc[index].text == 'they' and count7==0:
        about_inc=nlp(replace_with_error(about_inc,index,about_inc[index].text,'he'))
        count7=1
        

      #8. Run-on sentences,comma splices: The issue is highly [debatable, a → debatable. A] genetic risk
      elif about_inc[index].text.islower() ==  False and about_inc[index].text!='“' and str(about_inc[index-1]) == '.' \
      and not str(about_inc).startswith(about_inc[index].text) and not str(about_inc).endswith('.') and about_inc[index].text!='I' and count8==0:
        about_inc=nlp(replace_with_error(about_inc,index,about_inc[index-1].text+" "+about_inc[index].text,', '+about_inc[index].text.lower()))
        count8=1
        

      #9. Incorrect adjective/adverb order: In conclusion, [personally I → I personally] feel that it
      elif token.tag_ == 'RB' and count9==0 and not "'" in about_inc[index].text and len(about_inc[index].text)>=2 and not str(about_inc).startswith(about_inc[index].text):
        punc=0
        for p in string.punctuation:
          if p not in about_inc[index].text or p not in about_inc[index-1].text:
            punc=1
            break
        if(punc==0):
          about_inc=nlp(replace_with_error(about_inc,index,about_inc[index-1].text+" "+about_inc[index].text,about_inc[index].text+" "+about_inc[index-1].text))
          count9=1
          

      #10. Linking words/phrases: It is sometimes hard to find [out → out if] one has this disease.
      elif token.tag_ == 'IN' and j!=0 and count10==0:
        about_inc=nlp(replace_with_error(about_inc,index,about_inc[index].text+" ",""))
        count10=1
        
      #11. Poor citation
      elif (about_inc[index].text == '“') and count11==0:
        temp=str(about_inc)
        for word in temp.split():
          if word.startswith('“'):
            word1=word.replace('“','“ ')
            temp=temp.replace(word,word1)
          elif word.endswith('”'):
            word1=word.replace('”',' ”')
            temp=temp.replace(word,word1)
        if '“' in temp.split() and j!=temp.split().index('“') and not temp.endswith('”'):
          about_inc=nlp(replace_with_error(about_inc,index,about_inc[index].text,""))
          count11=1
        
    
      #13. Parallelism: We must pay attention to this information and [assisting → assist] those
      elif about_inc[index].text.endswith('ing') and about_inc[index-1].tag_ == 'CC' and about_inc[index].text!=token.lemma_ and about_inc[index].text[0].islower() and count13==0:
        about_inc=nlp(replace_with_error(about_inc,index,about_inc[index].text,token.lemma_))
        count13=1
        

      #14. Improper preposition forms: This essay will [discuss about → discuss] whether a carrier
      elif 'discuss' in about_inc[index].text and count14==0:
        about_inc=nlp(replace_with_error(about_inc,index,about_inc[index].text,about_inc[index].text+" "+'about'))
        count14=1
        
      elif about_inc[index].text=='through' and count14==0:
        about_inc=nlp(replace_with_error(about_inc,index,about_inc[index].text,'to'))
        count14=1
        
      elif about_inc[index].text=='into' and count14==0:
        about_inc=nlp(replace_with_error(about_inc,index,about_inc[index].text,'in'))
        count14=1
        
      elif about_inc[index].text=='since' and count14==0:
        about_inc=nlp(replace_with_error(about_inc,index,about_inc[index].text,'for'))
        count14=1
        

      #15. Removing word contractions: I've-->I, we'll-->we, let's-->let
      elif about_inc[index].text.endswith(words_ends_with_cont) and count15==0:
        if about_inc[index].text == "n't":
          about_inc=nlp(replace_with_error(about_inc,index,"'t",""))
        else:
          about_inc=nlp(replace_with_error(about_inc,index,about_inc[index].text,""))
        count15=1
        
      
      #16. upper to lower, lower to upper:
      elif len(str(about_inc).split())<=2 and count16==0:
        if about_inc[index].text.isupper():
          about_inc=nlp(replace_with_error(about_inc,index,about_inc[index].text,about_inc[index].text.lower()))
          count16=1
          
        else:
          about_inc=nlp(replace_with_error(about_inc,index,about_inc[index].text,about_inc[index].text.upper()))
          count16=1
          

      #12. Incorrect Spelling
      elif len(about_inc[index].text)>2 and about_inc[index].tag_ != 'WP' and about_inc[index].tag_ != 'NNP' and "'" not in about_inc[index].text and "'" not in about_inc[index+1].text and count12==0:
        if "'" not in about_inc[index-1].text or str(about_inc).startswith(about_inc[index].text):
          #swapping letters for wrong spelling
          c1=about_inc[index].text[1]
          c2=about_inc[index].text[2]
          c0=about_inc[index].text[0]
          if c1!=c2:
            d=about_inc[index].text.replace(c2,"-",1)
            d=d.replace(c1,c2,1)
            d=d.replace(d[2],c1,1)
          else:
            d=about_inc[index].text.replace(c1,c0,1)
          about_inc=nlp(replace_with_error(about_inc,index,about_inc[index].text,d))
          count12=1
          

      #skipping tokens to avoid errors in three adjacent tokens
      if index>=len(about_inc):
        skip=2
      elif about_inc[index].text != about_doc[j].text:
        skip=2

    incorrect_lines.append(str(about_inc))

  return incorrect_lines

In [ ]:
nlp = spacy.load('en_core_web_sm')
words_ends_with_cont=("'ve","'ll","'s","'m","'d","'re","'t")
def seperating_punct_2(lines):
  final_lines=[]
  for line in tqdm(lines):
    line=line.replace('  ',' ')
    g=[]
    for token in nlp(line):
      if token.text.startswith('-') and len(token.text)>1:
        temp=token.text
        temp=temp.replace('-','- ')
        g.append(temp)
      elif token.text.startswith("'") and len(token.text)>1 and not token.text.endswith(words_ends_with_cont):
        temp=token.text
        temp=temp.replace("'","' ")
        g.append(temp)
      else:
        g.append(token.text)
    out=' '.join(g)
    final_lines.append(out)
    
  return final_lines

In [ ]:
train_lines=get_incorrect_lines(train['lines'])

In [ ]:
train['incorrect_lines']=seperating_punct_2(train_lines)
train['lines']=seperating_punct_2(train['lines'])

In [ ]:
indices=[]
for i,val in enumerate(train['lines']==train['incorrect_lines']):
  if(val==True):
    indices.append(i)

In [ ]:
len(indices)

91

In [ ]:
train=train.drop(train.index[indices])

In [ ]:
train.head()

,lines,incorrect_lines
3135,"“ Oh yeah , you 're right , ” said Ron , teari...","“ Oh yaeh , you 're right , ” say Ron , tearin..."
17240,As there was no proof that the Riddles had bee...,As tehre was no proof the Riddles have been mu...
41599,The head glowed blue and trembled noisily agai...,Teh head glowed blue and tremble noisily again...
9739,"“ You wo n't be needing it , ” he said .","“ Yuo wo n't be needing it , ” he say ."
11302,"“ Harry , swear to me you wo n't go looking fo...","“ Harry , sewar to me you won go looking Blac , ”"


In [ ]:
train.shape

(30869, 2)

In [ ]:
cv_lines=get_incorrect_lines(cv['lines'])

In [ ]:
cv['incorrect_lines']=seperating_punct_2(cv_lines)
cv['lines']=seperating_punct_2(cv['lines'])

In [ ]:
indices=[]
for i,val in enumerate(cv['lines']==cv['incorrect_lines']):
  if(val==True):
    indices.append(i)

In [ ]:
len(indices)

19

In [ ]:
cv=cv.drop(cv.index[indices])

In [ ]:
cv.head()

,lines,incorrect_lines
17071,It took a moment for Harry to realize what Dum...,It take a moment Harry to raelize what Dumbled...
21655,Harry knocked on the door and entered .,Harry knock on the ddor and entered .
31864,"“ Still , it wo n't hurt to have some new bloo...","“ Still , it won hurt to hvae some new blood ,..."
29234,“ Where are we ? ”,“ Wehre are we ? ”
403,“ It was only a dream . ”,“ It be only a deram . ”


In [ ]:
cv.shape

(7721, 2)

In [ ]:
test_lines=get_incorrect_lines(test['lines'])

In [ ]:
test['incorrect_lines']=seperating_punct_2(test_lines)
test['lines']=seperating_punct_2(test['lines'])

In [ ]:
indices=[]
for i,val in enumerate(test['lines']==test['incorrect_lines']):
  if(val==True):
    indices.append(i)

In [ ]:
len(indices)

19

In [ ]:
test=test.drop(test.index[indices])

In [ ]:
test.head()

,lines,incorrect_lines
47215,Harry said nothing .,Harry say nothing .
6467,“ What 've we got this afternoon ? ” said Harr...,"“ What we got tihs afternoon ? ” say Harry , h..."
43348,“ Oh yes you did . You showed Horace exactly h...,"“ Oh yse you did , you showed Horace exactly h..."
48236,He had the solution at last : There was a way ...,He have the solution last : Tehre was a way to...
15302,“ Wan ' some tea ? ” he said .,“ Wan ' some tae ? ” he say .


In [ ]:
train.to_csv('train.csv',index=False)
cv.to_csv('cv.csv',index=False)
test.to_csv('test.csv',index=False)

In [ ]:
train=pd.read_csv('train.csv')
cv=pd.read_csv('cv.csv')
test=pd.read_csv('test.csv')